# Spark Structured Streaming and Delta Tables

Spark provides support for streaming data through *Spark Structured Streaming* and extends this support through *delta tables* that can be targets (*sinks*) or *sources* of streaming data.

In this exercise, you'll use Spark to ingest a stream of data from a folder of JSON files that consists of simulated status messages from devices. In a real scenario, the data could come from some other real-time source, such as a Kafka queue or an Azure Event Hub.

## Create a folder for the incoming stream of data

1. Ensure this notebook is attached to your Spark pool (using this **Attach to** drop-down list above).
2. Run the cell below to create a folder named **data** to which the simulated device data will be written.

    > **Note**: The first cell may take some time to run because the Spark pool must be started.


In [1]:
from notebookutils import mssparkutils

# Create a folder
inputPath = '/data/'
mssparkutils.fs.mkdirs(inputPath)

StatementMeta(sparkpool0110, 0, 2, Finished, Available, Finished)

True

## Use Spark Structured Streaming to query a stream of data

1. Run the cell below to create a streaming dataframe that reads data from the folder based on a JSON schema that includes the name of the device and its status.

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
  StructField("device", StringType(), False),
  StructField("status", StringType(), False)
])

fileDF = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)



StatementMeta(sparkpool0110, 0, 3, Finished, Available, Finished)

2. Wait for the cell above to complete.
3. When the streaming dataframe has been created, you can apply a transformation query to aggregate the data and write the results to an output stream. Run the following code to filter the incoming stream for errors in the device data, and count the number of errors per device.

In [3]:
countDF = fileDF.filter("status == 'error'").groupBy("device").count()
query = countDF.writeStream.format("memory").queryName("counts").outputMode("complete").start()
print('Streaming query started.')

StatementMeta(sparkpool0110, 0, 4, Finished, Available, Finished)

Streaming query started.


4. The query output is streamed to an in-memory table. Run the cell below to use SQL to query this table and veiw the number of errors per device.

In [5]:
%%sql
select * from counts


StatementMeta(sparkpool0110, 0, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 2 fields>

5. Note that the query returns no data, because we haven't written any device status data there yet.
6. Let's fix that by writing some status event data from a couple of simulated devices.

In [6]:
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data, True)

StatementMeta(sparkpool0110, 0, 7, Finished, Available, Finished)

True

7. Run the SQL query again to see the aggregated error counts (if the query still returns no data, wait a few seconds and try again!) There should be one error for device 1, and two errors for device 2.

In [9]:
%%sql
select * from counts


StatementMeta(sparkpool0110, 0, 10, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 2 fields>

8. Review the results, noting the number of errors. Then run the following code to write more device data.

In [10]:
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(sparkpool0110, 0, 11, Finished, Available, Finished)

True

9. Run the SQL query again (waiting a few seconds if necessary) to see the new status events reflected in the aggregations. There should now be two errors for device 1, and three errors for device 2.

In [11]:
%%sql
select * from counts


StatementMeta(sparkpool0110, 0, 12, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 2 fields>

## Create a delta table

Azure Synapse Analytics supports the Linux Foundation *Delta Lake* architecture, which builds on Spark Structured Streaming to add support for transactions, versioning, and other useful capabilities.

In particular, you can create *delta tables* as a target (or *sink*) for streaming data, or as a *source* of streaming data for downstream queries.

To explore this, we'll write the streaming dataframe based on the **data** folder we created previously to a new delta table, which we'll define using a path to a location in the file system.

1. Run the cell below to stream the folder data to a delta table.

In [12]:
delta_table_path = inputPath + 'deltatable'
stream = fileDF.writeStream.format("delta").option("checkpointLocation", inputPath + 'checkpoint').start(delta_table_path)

StatementMeta(sparkpool0110, 0, 13, Finished, Available, Finished)

2. Now run the next cell to query the delta table to see the data that has been streamed to it. If at first the query returns no data, wait a few seconds and run the cell again).

In [13]:
df = spark.read.format("delta").load(delta_table_path)
display(df)

StatementMeta(sparkpool0110, 0, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a0b2dc92-abb5-457d-a2d5-03536f1e5eaa)

Delta tables enable you to use a feature named *time travel* to view the data at a previous point in time.

4. Run the following query to retrieve the initial micro-batch of data that was streamed from the file data. 

In [14]:
df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(df)

StatementMeta(sparkpool0110, 0, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 864bceaf-3348-4181-a93f-19241ef21999)

5. Now that you've finished exploring Spark Structured Streaming and delta tables, stop the stream of data and clean up the files used in this exercise.

In [15]:
stream.stop()
query.stop()
print("Stream stopped")
mssparkutils.fs.rm(inputPath, True)

StatementMeta(sparkpool0110, 0, 16, Finished, Available, Finished)

Stream stopped


True